In [8]:
#### GO TO THE NEXT CHUNK OF CODE FOR THE "FINAL VERSION OF THE MODEL," RAN BY THE FUNCTION "BMT"


#Assume that number of nurses and spaces are fixed 
#for D1 and D2

#Jul 24 Morning
    #Removed the doctor decision variable MDi[i] completely.
        #anything i want to do with that decision can be replaced with the binary myDocID[i,d]
#Jul 24 Evening
    #randomly generate collections and doctor availability more like what Yu-Li suggested.
    #changing nursing capacity into a hard constraint as Yu-Li suggested.
#Jul 25 Afternoon
    # The D2 space and nurse constraints have been multiplied by 8.
        # Why?: any of the P[1] nurses available for D2, they can be work 8 D2's in a day.
#Jul 25 Evening
    # Similarly, MD's cannot see more than 8 patients in a day.
    # Reorganized the parameters/accompanying comments so they are more readable and clearer.
#Jul 26 Morning
    # Began implementing chemo. 
        
        #BIG CHANGE: rehauled the entire code's day index j be based on a 7 day week.
            #why?: i'm just too lazy to figure out how to convert a 5 day index a 7 day index halfway into the code
        
            #created a wkdayjbin binary that is 1 if and only if it is a weekday.
            #the D1ij and D2ijd binaries now are bounded above by the weekday identifier binary
                #thus, it will be one if the patient could be scheduled AND it is a weekday.
            #now that j is based on a 7 day week, D1ij now factors in weekends in its computation
                #created D1endi to compute the TRUE end date of stem cell collections
            #the daily sigma computations are now multiplied by the wkdayj binary
                #thus, having no prechemo/stem cell collections on a weekend won't contribute to sum of sigmas
        
        #New parameter: Chem[i] = number of chemo days needed for patient i
            #so, D2[i]+Chem[i] = the last day of chemo
            #constraint: D2[i]+Chem[i] cannot be Thursday or Friday
                #since transfusion can't occur on a weekend
            #as long as ^ is satisfied I won't assume any other scheduling hurdles for now
                #[i.e. space and nurse constraints for chem or infusion]
#Jul 27 Morning
    #Rewrote chem collection to be randomly sampled from the triangular distribution Yu-Li suggested.
    #Cleaned up comments.
    
#Jul 28 Morning
    #Revised initialization of P, S, anticipating Penny's eventual use of them in her rolling horizon.
    #Add ChemCap and InfCap, daily capacities for Chemo and Infusion respectively.
        
import gurobipy as gp

import numpy as np

###Parameters

    #number of patients
pat = 40

    #number of days for scheduling
tend = 28
    
    #randomly sample collection amounts for each patient
C = [np.random.choice([1,2,3,4,5], p =[.398,.304,.147,.088,.063]) for i in range(pat)]

    #number of chemo days needed for patient i
Chem = [int(np.around(np.random.triangular(1,2,6))) for i in range(pat)]

    #P[k] = number of nurses available for Dk
P = [10,4] 
P1 = [10 for j in range(tend)]
P2 = [4 for j in range(tend)]
    
    #S[k] number of spaces available for Dk
S = [15,4]
S1 = [15 for j in range(tend)]
S2 = [4 for j in range(tend)]
ChemCap = [8 for j in range(tend)]
InfCap = [5 for j in range(tend)]


    #W[k] = target number of patients in any given days for Dk
W = [7,4] 

    #number of MDs available
docs = 24 

    #what is the day of the week doctor d is available?
weekdayavaild = [np.random.randint(0,5) for d in range(docs)] 

    #generate a list of available days for doctor d
availdj = [[int(j%7 == weekdayavaild[d]) for j in range(tend)]for d in range(docs)] 



    # Create model
model = gp.Model()

### Decision variables
D1 = model.addVars(len(C),lb = 0, ub = tend-1, vtype=gp.GRB.INTEGER, name='D1_start')
D2 = model.addVars(len(C),lb = 0, ub = tend-1, vtype=gp.GRB.INTEGER, name='D2')
myDocID = model.addVars(pat,docs,vtype=gp.GRB.BINARY,name="doctor_binary")

### Objective function
    #define varibles       
        # vector of 1 during days stem cell collections occur for patient i on day j.
D1ij = model.addVars(pat,tend,vtype=gp.GRB.BINARY,name="stem_cell_binary")

        # vector of 1 EXACTLY when prechemo occurs for patient i with doctor d on day j.
D2ijd = model.addVars(pat,tend,docs,vtype=gp.GRB.BINARY,name="prechemo_binary")
        
        # totaling how many patients are seen in a day for a procedure
T1j = model.addVars(tend,vtype=gp.GRB.INTEGER,name="total_stem_cells_day_j")
T2j = model.addVars(tend,vtype=gp.GRB.INTEGER,name="total_prechemo_day_j")
Tchemj = model.addVars(tend,vtype=gp.GRB.INTEGER,name="total_chemo_day_j")
Tinfj = model.addVars(tend,vtype=gp.GRB.INTEGER,name="total_infusions_day_j")
        
        # target patient error
signsig1j = model.addVars(tend,lb=-2000,vtype=gp.GRB.INTEGER,name = "signed daily error 1")
signsig2j = model.addVars(tend,lb=-2000,vtype=gp.GRB.INTEGER,name = "signed daily error 2")

sig1j = model.addVars(tend,vtype=gp.GRB.INTEGER,name="daily error 1")
sig2j = model.addVars(tend,vtype=gp.GRB.INTEGER,name="daily error 2")

        # total of target patient error
sig1 = model.addVar(vtype=gp.GRB.INTEGER,name = "total error 1")
sig2 = model.addVar(vtype=gp.GRB.INTEGER,name = "total error 2")

        # Q1 gap
            #we need helper variables to make sure a weekend is accounted for
Qi = model.addVars(pat,vtype=gp.GRB.INTEGER,name="Q1")
Qtot = model.addVar(vtype=gp.GRB.INTEGER,name="Q1 sum")

        #Weekday binary creation variables
wkj = model.addVars(tend, vtype=gp.GRB.INTEGER,name = "week")
wkdayj = model.addVars(tend,ub=6, vtype=gp.GRB.INTEGER,name = "weekday")
wkdaybin = model.addVars(tend, vtype=gp.GRB.BINARY,name = "weekend binary")

        #Variables that compute the end date of stem cell collection
N1i = model.addVars(pat,vtype=gp.GRB.INTEGER,name="N1")
R1i = model.addVars(pat,ub = 4,vtype=gp.GRB.INTEGER,name="R1")
N2i = model.addVars(pat,vtype=gp.GRB.INTEGER,name="N2")
R2i = model.addVars(pat,ub = 6,vtype=gp.GRB.INTEGER,name="R2")
D1endi = model.addVars(pat,vtype=gp.GRB.INTEGER,name="D1end")

        # chemo binary
chemij = model.addVars(pat,tend,vtype=gp.GRB.BINARY, name = "chemo binary")

        # infusion day binary (works just like D1ij,D2ijd)
infbinij = model.addVars(pat,tend,vtype=gp.GRB.BINARY, name ="infusion binary")


### Constraints

    #goal: compute D1endi = last date of stem cell collections
model.addConstrs(D1[i]==7*N1i[i]+R1i[i] for i in range(pat))
model.addConstrs(R1i[i]+C[i]-1 == 7*N2i[i]+R2i[i] for i in range(pat))
model.addConstrs(D1endi[i]==D1[i]+C[i]-1+2*N2i[i] for i in range(pat))

    #goal: compute wkdaybin = what day of the week are we on?
model.addConstrs(j == (wkj[j]*7+wkdayj[j]) for j in range(tend))
model.addConstrs((wkdaybin[j]==1)>>(wkdayj[j]<=4) for j in range(tend))
model.addConstrs((wkdaybin[j]==0)>>(wkdayj[j]>=5) for j in range(tend))

    #goal: create D1ij
        #"if D1ij is true, then j is at least the collection date"
model.addConstrs((D1ij[i,j]==1)>>(j >= D1[i]) for i in range(pat) for j in range(tend))
        #"if D1ij is true, then j is at most the end date"
model.addConstrs((D1ij[i,j]==1)>>(j <= D1endi[i]) for i in range(pat) for j in range(tend))
        #"a patient gets as many collections as they actually need"
model.addConstrs(gp.quicksum([D1ij[i,j] for j in range(tend)]) == C[i] for i in range(pat))
        #"a patient cannot get collections on a weekend"
model.addConstrs((D1ij[i,j]<=wkdaybin[j]) for i in range(pat) for j in range(tend))    

    #constraint: every patient must have 1 MD
model.addConstrs(gp.quicksum(myDocID[i,d] for d in range(docs)) ==1  for i in range(pat))

    #goal: create D2ijd
        #"if D2ijd is true, then j is the prechemo date"
model.addConstrs((D2ijd[i,j,d]==1)>>(j == D2[i]) for i in range(pat) for j in range(tend) for d in range(docs))
        # a patient can be scheduled for D2 on a WEEKDAY when THEIR MD is AVAILABLE
model.addConstrs((D2ijd[i,j,d]<=wkdaybin[j]*myDocID[i,d]*availdj[d][j]) for i in range(pat) for j in range(tend) for d in range(docs))
        #"patients MUST have prechemo"
model.addConstrs(gp.quicksum([D2ijd[i,j,d] for j in range(tend) for d in range(docs)]) == 1 for i in range(pat))

    #goal: create chemij
        #"if chemij is true, then j is at least the start date of chemo"
model.addConstrs((chemij[i,j]==1)>>(j >= D2[i]+1) for i in range(pat) for j in range(tend))
        #"if chemij is true, then j is at most the end date"
model.addConstrs((chemij[i,j]==1)>>(j <= D2[i]+Chem[i]) for i in range(pat) for j in range(tend))
        #"a patient gets as many days in chemo as they actually need"
model.addConstrs(gp.quicksum([chemij[i,j] for j in range(tend)]) == Chem[i] for i in range(pat))

    #constraint: infusion happens after a day of rest from chemo and must happen on a weekday
        #subgoal: create infbinij
            #infusion happens after a day of rest from chemo
                #note that D2[i]+Chem[i]+2 is the tentative date of infusion
model.addConstrs((infbinij[i,j]==1)>>(j==(D2[i]+Chem[i]+2)) for i in range(pat) for j in range(tend))
            #infusion must happen on a weekday
model.addConstrs(infbinij[i,j]<=wkdaybin[j] for i in range(pat) for j in range(tend))
            #infusion must happen
model.addConstrs(gp.quicksum(infbinij[i,j] for j in range(tend))==1 for i in range(pat))

    #goal: compute D1 daily counts
model.addConstrs(T1j[j]==gp.quicksum([D1ij[i,j] for i in range(pat)]) for j in range(tend))

    #goal: compute D2 daily counts
model.addConstrs(T2j[j]==gp.quicksum([D2ijd[i,j,d] for i in range(pat) for d in range(docs)]) for j in range(tend))

    #goal: compute chemo daily counts
model.addConstrs(Tchemj[j]==gp.quicksum([chemij[i,j] for i in range(pat)]) for j in range(tend))

    #goal: compute infusion daily counts
model.addConstrs(Tinfj[j]==gp.quicksum([infbinij[i,j] for i in range(pat)]) for j in range(tend))


    #constraint: D2 only happens after D1 finishes
model.addConstrs((D1endi[i])<=(D2[i]-1) for i in range(pat))

    #constraint: we cannot have more patients than spaces for each procedure
model.addConstrs(T1j[j]<=S1[j] for j in range(tend))
model.addConstrs(T2j[j]<=8*S2[j] for j in range(tend))
model.addConstrs(Tchemj[j]<=ChemCap[j] for j in range(tend))
model.addConstrs(Tinfj[j]<=InfCap[j] for j in range(tend))

    #constraint: we cannot have more patients than nurses for each procedure
model.addConstrs(T1j[j]<=P1[j] for j in range(tend))
model.addConstrs((T2j[j]<=8*P2[j]) for j in range (tend))

    #constraint: MD can see at most 8 patients in a given day for D2. 
        #[8 hour work day / 1 hour per appt]
model.addConstrs(gp.quicksum(D2ijd[i,j,d]for i in range(pat) for j in range(tend)) <= 8 for d in range(docs))

    #goal: calculate error
        #sig_1
            #create "signsig1j" be the difference of T1j and W0
model.addConstrs(signsig1j[j]==(T1j[j]-W[0])*wkdaybin[j] for j in range(tend))
            #let "sig1j" be the absolute value of signsig1j
model.addConstrs(sig1j[j]==gp.abs_(signsig1j[j]) for j in range(tend))
            #sum up all sig1j's
model.addConstr(sig1 == gp.quicksum([sig1j[j] for j in range(tend)]))

        #sig_2
model.addConstrs(signsig2j[j]==(T2j[j]-W[1])*wkdaybin[j] for j in range(tend))
model.addConstrs(sig2j[j]==gp.abs_(signsig2j[j]) for j in range(tend))
model.addConstr(sig2 == gp.quicksum([sig2j[j] for j in range(tend)]))

    #goal: calculate Q
model.addConstrs(Qi[i]==(D2[i]-D1endi[i]) for i in range(pat))
model.addConstr(Qtot == gp.quicksum(Qi[i] for i in range(pat)))


model.update()

#setObjective
model.setObjective(Qtot+sig1+sig2,gp.GRB.MINIMIZE) 

#optimize the model
model.optimize()

# check the status to see if there is a solution
status = model.getAttr("Status")
if status == 2:
    for j in range(len(C)):
        print("Patient "+str(j)+" schedule (D1,D2):(", D1[j].getAttr("X"), D2[j].getAttr("X"),")")


Using license file /Users/Skye/gurobi.lic
Academic license - for non-commercial use only


KeyboardInterrupt: 

In [30]:
#Jul 29 Changes
    #roll now takes in an optional target patient number argument
        #due to previous comments from ankit and yu li, it will default to the capacities list if left empty
        #i've changed the argument order to make this more intuitive
        #BMT now computes the daily target error for chemo and infusion like D1 and D2
    #some typo-related bugs fixed
    #more comments!


import gurobipy as gp

import numpy as np

def roll(ndays,docs,caps,targs=None):
        #ndays = number of days to simulate
        #caps = capacities [StemCap,PChemCap,ChemCap,InfCap]
        #docs = number of MDs
        #targs = target numbers [targetStem,targetPChem,targetChem,targetInf]
            #defaults to caps unless specified
        #output is the last schedule made [100% confident it will be because we make it impossible to schedule chemo!]
        
        #set target list to capacity list if unspecified
        if targs==None:
            targs=caps
            
        #total patients in scheduled for each of the 4 procedures
            #currently none, but updates as BMT loops
        tots = [[0 for i in range(ndays)] for i in range(4)]
        
        #current list of patients entered and scheduled
            #currently empty, but updates as BMT loops
        schedules = []
        
            #what is the day of the week doctor d is available?
        weekdayavaild = [np.random.randint(0,5) for d in range(docs)] 
        
        for day in range(ndays):
            print ("DAY " + str(day))
            pat = 2
            run = BMT(day,ndays,pat,tots,caps,targs,weekdayavaild)
            if run!=False:        
                tots = np.add(tots,run[0])
                schedules+=run[1]
                print("Patient's D1, D2, and Q1")
                print(schedules)
                print ("Current number of patients in stem cell collection each day")
                print(tots[0])
                print ("Current number of patients in prechemo each day")
                print(tots[1])
                print("Current number of patients in chemo each day")
                print(tots[2])
                print("Current number of patients in infusion each day")
                print(tots[3])
            else:
                #looks like this model becomes infeasible at some point before the end date of simulation
                    #this is due to the indices we've limited scheduling the later processes of the BMT process
                    #we can think of ways to tweak this later
                        #however, i think it'd be equivalent and suffices to just overshoot the ndays
                print("cannot schedule more")
                return [tots,schedules]
        return [tots,schedules]


def BMT(tstart,tend,pat,tots,caps,targs,weekdayavaild):
        #capacities
    docs=len(weekdayavaild)
    StemCap = caps[0]
    PreChemCap = caps[1]
    ChemCap = caps[2] 
    InfCap = caps[3]
    W = targs
        #randomly sample collection amounts for each patient
    C = [np.random.choice([1,2,3,4,5], p =[.398,.304,.147,.088,.063]) for i in range(pat)]

        #number of chemo days needed for patient i
    Chem = [int(np.around(np.random.triangular(1,2,6))) for i in range(pat)]







        #generate a list of available days for doctor d
    availdj = [[int(j%7 == weekdayavaild[d]) for j in range(tend)]for d in range(docs)] 

        # Create model
    model = gp.Model()

    ### Decision variables
    D1 = model.addVars(len(C),lb = tstart, ub = tend-1, vtype=gp.GRB.INTEGER, name='D1_start')
    D2 = model.addVars(len(C),lb = 0, vtype=gp.GRB.INTEGER, name='D2')
    myDocID = model.addVars(pat,docs,vtype=gp.GRB.BINARY,name="doctor_binary")

    ### Objective function
        #define varibles       
            # vector of 1 during days stem cell collections occur for patient i on day j.
    D1ij = model.addVars(pat,tend,vtype=gp.GRB.BINARY,name="stem_cell_binary")

            # vector of 1 EXACTLY when prechemo occurs for patient i with doctor d on day j.
    D2ijd = model.addVars(pat,tend,docs,vtype=gp.GRB.BINARY,name="prechemo_binary")

            # totaling how many patients are seen in a day for a procedure
    T1j = model.addVars(tend,vtype=gp.GRB.INTEGER,name="total_stem_cells_day_j")
    T2j = model.addVars(tend,vtype=gp.GRB.INTEGER,name="total_prechemo_day_j")
    Tchemj = model.addVars(tend,vtype=gp.GRB.INTEGER,name="total_chemo_day_j")
    Tinfj = model.addVars(tend,vtype=gp.GRB.INTEGER,name="total_infusions_day_j")

            # target patient error
    signsig1j = model.addVars(tend,lb=-2000,vtype=gp.GRB.INTEGER,name = "signed daily error 1")
    signsig2j = model.addVars(tend,lb=-2000,vtype=gp.GRB.INTEGER,name = "signed daily error 2")
    signsig3j = model.addVars(tend,lb=-2000,vtype=gp.GRB.INTEGER,name = "signed daily error 3")
    signsig4j = model.addVars(tend,lb=-2000,vtype=gp.GRB.INTEGER,name = "signed daily error 4")

    sig1j = model.addVars(tend,vtype=gp.GRB.INTEGER,name="daily error 1")
    sig2j = model.addVars(tend,vtype=gp.GRB.INTEGER,name="daily error 2")
    sig3j = model.addVars(tend,vtype=gp.GRB.INTEGER,name="daily error 3")
    sig4j = model.addVars(tend,vtype=gp.GRB.INTEGER,name="daily error 4")

            # total of target patient error
    sig1 = model.addVar(vtype=gp.GRB.INTEGER,name = "total error 1")
    sig2 = model.addVar(vtype=gp.GRB.INTEGER,name = "total error 2")
    sig3 = model.addVar(vtype=gp.GRB.INTEGER,name = "total error 3")
    sig4 = model.addVar(vtype=gp.GRB.INTEGER,name = "total error 4")
            # Q1 gap
                #we need helper variables to make sure a weekend is accounted for
    Qi = model.addVars(pat,vtype=gp.GRB.INTEGER,name="Q1")
    Qtot = model.addVar(vtype=gp.GRB.INTEGER,name="Q1 sum")

            #Weekday binary creation variables
    wkj = model.addVars(tend, vtype=gp.GRB.INTEGER,name = "week")
    wkdayj = model.addVars(tend,ub=6, vtype=gp.GRB.INTEGER,name = "weekday")
    wkdaybin = model.addVars(tend, vtype=gp.GRB.BINARY,name = "weekend binary")

            #Variables that compute the end date of stem cell collection
    N1i = model.addVars(pat,vtype=gp.GRB.INTEGER,name="N1")
    R1i = model.addVars(pat,ub = 4,vtype=gp.GRB.INTEGER,name="R1")
    N2i = model.addVars(pat,vtype=gp.GRB.INTEGER,name="N2")
    R2i = model.addVars(pat,ub = 6,vtype=gp.GRB.INTEGER,name="R2")
    D1endi = model.addVars(pat,vtype=gp.GRB.INTEGER,name="D1end")

            # chemo binary
    chemij = model.addVars(pat,tend,vtype=gp.GRB.BINARY, name = "chemo binary")

            # infusion day binary (works just like D1ij,D2ijd)
    infbinij = model.addVars(pat,tend,vtype=gp.GRB.BINARY, name ="infusion binary")


    ### Constraints

        #goal: compute D1endi = last date of stem cell collections
    model.addConstrs(D1[i]==7*N1i[i]+R1i[i] for i in range(pat))
    model.addConstrs(R1i[i]+C[i]-1 == 7*N2i[i]+R2i[i] for i in range(pat))
    model.addConstrs(D1endi[i]==D1[i]+C[i]-1+2*N2i[i] for i in range(pat))

        #goal: compute wkdaybin = what day of the week are we on?
    model.addConstrs(j == (wkj[j]*7+wkdayj[j]) for j in range(tend))
    model.addConstrs((wkdaybin[j]==1)>>(wkdayj[j]<=4) for j in range(tend))
    model.addConstrs((wkdaybin[j]==0)>>(wkdayj[j]>=5) for j in range(tend))

        #goal: create D1ij
            #"if D1ij is true, then j is at least the collection date"
    model.addConstrs((D1ij[i,j]==1)>>(j >= D1[i]) for i in range(pat) for j in range(tend))
            #"if D1ij is true, then j is at most the end date"
    model.addConstrs((D1ij[i,j]==1)>>(j <= D1endi[i]) for i in range(pat) for j in range(tend))
            #"a patient gets as many collections as they actually need"
    model.addConstrs(gp.quicksum([D1ij[i,j] for j in range(tend)]) == C[i] for i in range(pat))
            #"a patient cannot get collections on a weekend"
    model.addConstrs((D1ij[i,j]<=wkdaybin[j]) for i in range(pat) for j in range(tend))    

        #constraint: every patient must have 1 MD
    model.addConstrs(gp.quicksum(myDocID[i,d] for d in range(docs)) ==1  for i in range(pat))

        #goal: create D2ijd
            #"if D2ijd is true, then j is the prechemo date"
    model.addConstrs((D2ijd[i,j,d]==1)>>(j == D2[i]) for i in range(pat) for j in range(tend) for d in range(docs))
            # a patient can be scheduled for D2 on a WEEKDAY when THEIR MD is AVAILABLE
    model.addConstrs((D2ijd[i,j,d]<=wkdaybin[j]*myDocID[i,d]*availdj[d][j]) for i in range(pat) for j in range(tend) for d in range(docs))
            #"patients MUST have prechemo"
    model.addConstrs(gp.quicksum([D2ijd[i,j,d] for j in range(tend) for d in range(docs)]) == 1 for i in range(pat))

        #goal: create chemij
            #"if chemij is true, then j is at least the start date of chemo"
    model.addConstrs((chemij[i,j]==1)>>(j >= D2[i]+1) for i in range(pat) for j in range(tend))
            #"if chemij is true, then j is at most the end date"
    model.addConstrs((chemij[i,j]==1)>>(j <= D2[i]+Chem[i]) for i in range(pat) for j in range(tend))
            #"a patient gets as many days in chemo as they actually need"
    model.addConstrs(gp.quicksum([chemij[i,j] for j in range(tend)]) == Chem[i] for i in range(pat))

        #constraint: infusion happens after a day of rest from chemo and must happen on a weekday
            #subgoal: create infbinij
                #infusion happens after a day of rest from chemo
                    #note that D2[i]+Chem[i]+2 is the tentative date of infusion
    model.addConstrs((infbinij[i,j]==1)>>(j==(D2[i]+Chem[i]+2)) for i in range(pat) for j in range(tend))
                #infusion must happen on a weekday
    model.addConstrs(infbinij[i,j]<=wkdaybin[j] for i in range(pat) for j in range(tend))
                #infusion must happen
    model.addConstrs(gp.quicksum(infbinij[i,j] for j in range(tend))==1 for i in range(pat))

        #goal: compute D1 daily counts
    model.addConstrs(T1j[j]==gp.quicksum([D1ij[i,j] for i in range(pat)]) for j in range(tend))

        #goal: compute D2 daily counts
    model.addConstrs(T2j[j]==gp.quicksum([D2ijd[i,j,d] for i in range(pat) for d in range(docs)]) for j in range(tend))

        #goal: compute chemo daily counts
    model.addConstrs(Tchemj[j]==gp.quicksum([chemij[i,j] for i in range(pat)]) for j in range(tend))

        #goal: compute infusion daily counts
    model.addConstrs(Tinfj[j]==gp.quicksum([infbinij[i,j] for i in range(pat)]) for j in range(tend))


        #constraint: D2 only happens after D1 finishes
    model.addConstrs((D1endi[i])<=(D2[i]-1) for i in range(pat))

        #constraint: we cannot have more patients than spaces for each procedure
    model.addConstrs(tots[0][j]+T1j[j]<=StemCap for j in range(tend))
    model.addConstrs(tots[1][j]+T2j[j]<=8*PreChemCap for j in range(tend))
    model.addConstrs(tots[2][j]+Tchemj[j]<=ChemCap for j in range(tend))
    model.addConstrs(tots[3][j]+Tinfj[j]<=InfCap for j in range(tend))

        #constraint: MD can see at most 8 patients in a given day for D2. 
            #[8 hour work day / 1 hour per appt]
    model.addConstrs(gp.quicksum(D2ijd[i,j,d]for i in range(pat) for j in range(tend)) <= 8 for d in range(docs))

        #goal: calculate error
            #sig_1
                #create "signsig1j" be the difference of T1j and W0
    model.addConstrs(signsig1j[j]==(tots[0][j]+T1j[j]-W[0])*wkdaybin[j] for j in range(tend))
                #let "sig1j" be the absolute value of signsig1j
    model.addConstrs(sig1j[j]==gp.abs_(signsig1j[j]) for j in range(tend))
                #sum up all sig1j's
    model.addConstr(sig1 == gp.quicksum([sig1j[j] for j in range(tend)]))

            #sig_2
    model.addConstrs(signsig2j[j]==(tots[1][j]+T2j[j]-W[1])*wkdaybin[j] for j in range(tend))
    model.addConstrs(sig2j[j]==gp.abs_(signsig2j[j]) for j in range(tend))
    model.addConstr(sig2 == gp.quicksum([sig2j[j] for j in range(tend)]))
            
            #sig_3
    model.addConstrs(signsig3j[j]==(tots[2][j]+Tchemj[j]-W[2]) for j in range(tend))
    model.addConstrs(sig3j[j]==gp.abs_(signsig3j[j]) for j in range(tend))
    model.addConstr(sig3 == gp.quicksum([sig3j[j] for j in range(tend)]))

            #sig_4
    model.addConstrs(signsig4j[j]==(tots[3][j]+Tinfj[j]-W[3]) for j in range(tend))
    model.addConstrs(sig4j[j]==gp.abs_(signsig4j[j]) for j in range(tend))
    model.addConstr(sig4 == gp.quicksum([sig4j[j] for j in range(tend)]))

        #goal: calculate Q
    model.addConstrs(Qi[i]==(D2[i]-D1endi[i]) for i in range(pat))
    model.addConstr(Qtot == gp.quicksum(Qi[i] for i in range(pat)))


    model.update()

    #setObjective
    model.setObjective(gp.quicksum((D1[i]-tstart) for i in range(pat))+4*Qtot+10*(sig1+sig2+sig3+sig4),gp.GRB.MINIMIZE) 

    #optimize the model
    model.optimize()
    if model.status == 2:
        #return [[Stem schedule,Prechem schedule,Chem schedule, Infusion schedule],[new patient's D1, D2 and Q1]]
        return [[[np.around(T1j[j].getAttr("X")) for j in range(tend)],[np.around(T2j[j].getAttr("X")) for j in range(tend)],[np.around(Tchemj[j].getAttr("X")) for j in range(tend)],[np.around(Tinfj[j].getAttr("X")) for j in range(tend)]],[[np.around(D1[i].getAttr("X")),np.around(D2[i].getAttr("X")),np.around(Qi[i].getAttr("X"))] for i in range(pat)]]
    else:
        #return false to break the simulaton loop
        return False

In [31]:
roll(200,20,[10,4,8,5],[7,4,7,3])

DAY 0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 3045 rows, 12261 columns and 31675 nonzeros
Model fingerprint: 0xc8d7930e
Model has 8400 quadratic constraints
Model has 11200 general constraints
Variable types: 0 continuous, 12261 integer (9440 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 7e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 2e+03]
  RHS range        [1e+00, 2e+02]
Presolve removed 217 rows and 8305 columns
Presolve time: 0.31s
Presolved: 2828 rows, 3956 columns, 8838 nonzeros
Presolved model has 1800 SOS constraint(s)
Variable types: 0 continuous, 3956 integer (2119 binary)
Found heuristic solution: objective 35688.000000

Root relaxation: cutoff, 102 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  

Model fingerprint: 0xb9a9435f
Model has 8400 quadratic constraints
Model has 11200 general constraints
Variable types: 0 continuous, 12261 integer (9440 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 7e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 2e+03]
  RHS range        [1e+00, 2e+02]
Presolve removed 1016 rows and 9624 columns
Presolve time: 0.91s
Presolved: 2029 rows, 2637 columns, 6053 nonzeros
Presolved model has 1157 SOS constraint(s)
Variable types: 0 continuous, 2637 integer (1447 binary)
Found heuristic solution: objective 35418.000000

Root relaxation: cutoff, 84 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      35418.0000 35418.0000  0.00%     -    1s

Explored 0 nodes (84 simplex iterations) in 1.04 seconds
Thread c

Model fingerprint: 0x1eacee3f
Model has 8400 quadratic constraints
Model has 11200 general constraints
Variable types: 0 continuous, 12261 integer (9440 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 7e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 2e+03]
  RHS range        [1e+00, 2e+02]
Presolve removed 1048 rows and 9674 columns
Presolve time: 0.80s
Presolved: 1997 rows, 2587 columns, 5737 nonzeros
Presolved model has 1134 SOS constraint(s)
Variable types: 0 continuous, 2587 integer (1420 binary)

Root relaxation: objective 3.510890e+04, 517 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 35108.8974    0 1146          - 35108.8974      -     -    1s
H    0     0                    35124.000000 35108.8974  0.04%     -    1s
     0     0 35110.6228

KeyboardInterrupt: 

In [ ]:
#seed is 2019